# Airnow Common

Constants and functions common to the various Airnow scripts.

In [0]:
import os, dateutil, requests, datetime

from dateutil import rrule, tz, parser

In [0]:
class AirnowCommonInstance:
    AIRNOW_ROOT_URL = 'https://files.airnowtech.org/airnow/'
    DATA_DIRECTORY = '../../airnow-data'

    DAILY_AQI_DIRECTORY = DATA_DIRECTORY + '/daily-aqi'
    DAILY_AQI_DAT_DIRECTORY = DAILY_AQI_DIRECTORY + '/dat'
    DAILY_AQI_JSON_DIRECTORY = DAILY_AQI_DIRECTORY + '/json'

    HOURLY_AQI_DIRECTORY = DATA_DIRECTORY + '/hourly-aqi'
    HOURLY_AQI_DAT_DIRECTORY = HOURLY_AQI_DIRECTORY + '/dat'
    HOURLY_AQI_JSON_DIRECTORY = HOURLY_AQI_DIRECTORY + '/json'

    DAILY_VALUES_DIRECTORY = DATA_DIRECTORY + '/daily-values'
    DAILY_VALUES_DAT_DIRECTORY = DAILY_VALUES_DIRECTORY + '/dat'

    HOURLY_VALUES_DIRECTORY = DATA_DIRECTORY + '/hourly-values'
    HOURLY_VALUES_DAT_DIRECTORY = HOURLY_VALUES_DIRECTORY + '/dat'
    HOURLY_VALUES_UPLOADED_DIRECTORY = HOURLY_VALUES_DIRECTORY + '/uploaded-to-esdr'

    HIGHEST_FIVE_AQI_DIRECTORY = DATA_DIRECTORY + '/highest-five-aqi'
    HIGHEST_FIVE_AQI_DAT_DIRECTORY = HIGHEST_FIVE_AQI_DIRECTORY + '/dat'

    def directory_from_date(self, dt):
        return dt.strftime('%Y/%Y%m%d')

    def datetime2epoch(self, dt):
        return (dt - datetime.datetime(1970, 1, 1, tzinfo=tz.tzutc())).total_seconds()

    # Returns a tuple containing (True, message, HTTP status) if the file was actually mirrored (i.e. both newer than the current version, and successfully downloaded), returns a tuple containing (False, message, HTTP status) otherwise.
    def mirror_file_using_modtime(self, src_url, dest):
        headers = {}
        # If destination already exists, mirror only if newer
        try:
            filestat = os.stat(dest)
            date = time.strftime('%a, %d %b %Y %H:%M:%S GMT', time.gmtime(filestat.st_mtime))
            headers['If-Modified-Since'] = date
        except:
            pass

        response = requests.get(src_url, headers=headers)
        if response.status_code == 200:
            data = response.content

            server_modtime = dateutil.parser.parse(response.headers['Last-Modified'])
            server_modtime_epoch = self.datetime2epoch(server_modtime)
            tmp = dest + '.tmp' + str(os.getpid())
            os.makedirs(os.path.dirname(tmp), exist_ok=True)
            open(tmp, 'wb').write(data)
            os.rename(tmp, dest)
            os.utime(dest, (server_modtime_epoch, server_modtime_epoch))
            print('Wrote %d bytes to %s' % (len(data), dest))
            return (True, 'Successfully mirrored %s to %s (%d bytes)' % (src_url, dest, len(data)), response.status_code)
        elif response.status_code == 304:
            return (False, 'Local mirror of %s is up to date.  Skipping.' % (src_url), response.status_code)
        elif response.status_code == 404:
            return (False, 'File %s not found (HTTP %d). Skipping.' % (src_url, response.status_code), response.status_code)
        else:
            return (False, 'Received status code %d while fetching %s.  Skipping.' % (response.status_code, src_url), response.status_code)

    # Mirrors a file from the Airnow server specified by src, assumed to be a path relative to `AirnowCommon.AIRNOW_ROOT_URL`.  Returns True if the file was actually mirrored (i.e. both newer than the current version, and successfully downloaded), returns False otherwise.
    def mirror_airnow_file(self, src, dest):
        return self.mirror_file_using_modtime(AirnowCommon.AIRNOW_ROOT_URL + src, dest)


AirnowCommon = AirnowCommonInstance()